In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices()
tf.config.experimental.set_memory_growth(physical_devices[3], True)
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, MaxPooling2D, Flatten, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import cv2

In [2]:
INPUT_SHAPE = (224, 224, 3)

# creating objects of class obtained from opencv lib
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
# Defining the model
def get_model():
    x_input = Input(INPUT_SHAPE)
    pretrained_model = MobileNetV2(input_shape=INPUT_SHAPE, 
                                   alpha=1.0, 
                                   include_top=False,
                                   weights='imagenet',
                                   input_tensor=x_input) 
    pretrained_model.trainable = True
    x = tf.reduce_mean(pretrained_model.outputs[0], axis = [1, 2])
    x = Flatten()(x)
    x = Dense(512, kernel_initializer = "he_normal")(x)
    x = tf.nn.relu(x)
    x = Dropout(0.2)(x)
    x = Dense(256, kernel_initializer = "he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(128, kernel_initializer = "he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(64, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(64, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(16, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(8, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(1, kernel_initializer ="he_normal")(x)
    x = tf.nn.sigmoid(x)
    model = Model(x_input, x)
    return model

model = get_model()
                                  
model.load_weights(str(os.getcwd()) +"/weights.hdf5")



In [5]:

def detect(gray,frame):
    faces=face_cascade.detectMultiScale(gray, 1.3, 5)
    # faces return turple of x,y,w,h of face detected
    

    for (x,y,w,h) in faces:
        
        # when face is detected the region of interest is defined to check if the person has mask or not
        roi_frame = frame[y:y+h,x:x+w]
        
        roi_frame = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB)   #BGR to RGB
        roi_frame = tf.cast(roi_frame, dtype = tf.float32)/255.0  #Standardization
        roi_frame = tf.image.resize(roi_frame, (INPUT_SHAPE[0], INPUT_SHAPE[1]))
        roi_frame = tf.expand_dims(roi_frame, axis=0)
        
        category = model(roi_frame, training = False)           #CNN Prediction
        category = category.numpy()
        
        text = "not known"
        if category>0.5:
            color = (0, 0, 255)   #BGR in opencv
            text = "No Mask " 
            
        elif category<0.5:
            color = (0, 255, 0)
            text = "Mask " 
         
        # To Draw line and write mask or no mask
        cv2.rectangle(frame, (x, y), (x+w, y+h),color,2)
        cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    return frame

# for turning on webcam
video_capture= cv2.VideoCapture(0)


# infinite loop 
while True:
    _,frame = video_capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    canvas = detect(gray,frame)
    cv2.imshow('Video',canvas)
  
    if cv2.waitKey(1) & 0xFF == ord('q'):   #Press "q" to turn off the detection
        break
video_capture.release()
cv2.destroyAllWindows()
